In [ ]:
#!pip install tensorflow==1.9.0

In [ ]:
!pip freeze | grep tensorflow

In [ ]:
# pip install tensorflow==1.9.0rc2 coremltools==0.8 tensorflowjs==0.5.0
# pip install tf-nightly==1.10.0.dev20180707
# nano /usr/local/anaconda3/envs/tf2/lib/python3.6/site-packages/tensorflow/contrib/lite/python/lite.py

In [ ]:
#pip install coremltools==0.8 Keras==2.1.6 imutils==0.4.6 opencv-python==3.4.1.15


In [ ]:
#https://github.com/ianlokh/MobileNet-On-CoreML.git

In [1]:
import os
import shutil
import hashlib
import urllib.request
import numpy as np
import json
#import keras
import tensorflow as tf
#import coremltools
import tensorflowjs as tfjs
from tensorflow.python import keras as tf_keras

/usr/local/anaconda3/envs/tf2/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:

#from keras import backend as K
#from keras.models import load_model
#from keras.utils.generic_utils import CustomObjectScope
#from keras.applications.mobilenet import preprocess_input

#from keras.preprocessing import image
#from keras.applications.mobilenet import preprocess_input



In [ ]:
!ls

In [2]:
with open('labels.txt') as f:
    labels = f.read().splitlines()

In [3]:
labels_dict = {idx:label for (idx,label) in enumerate(labels)}

In [4]:
labels_dict

{0: 'affenpinscher',
 1: 'afghan_hound',
 2: 'african_hunting_dog',
 3: 'airedale',
 4: 'american_staffordshire_terrier',
 5: 'appenzeller',
 6: 'australian_terrier',
 7: 'basenji',
 8: 'basset',
 9: 'beagle',
 10: 'bedlington_terrier',
 11: 'bernese_mountain_dog',
 12: 'black-and-tan_coonhound',
 13: 'blenheim_spaniel',
 14: 'bloodhound',
 15: 'bluetick',
 16: 'border_collie',
 17: 'border_terrier',
 18: 'borzoi',
 19: 'boston_bull',
 20: 'bouvier_des_flandres',
 21: 'boxer',
 22: 'brabancon_griffon',
 23: 'briard',
 24: 'brittany_spaniel',
 25: 'bull_mastiff',
 26: 'cairn',
 27: 'cardigan',
 28: 'chesapeake_bay_retriever',
 29: 'chihuahua',
 30: 'chow',
 31: 'clumber',
 32: 'cocker_spaniel',
 33: 'collie',
 34: 'curly-coated_retriever',
 35: 'dandie_dinmont',
 36: 'dhole',
 37: 'dingo',
 38: 'doberman',
 39: 'english_foxhound',
 40: 'english_setter',
 41: 'english_springer',
 42: 'entlebucher',
 43: 'eskimo_dog',
 44: 'flat-coated_retriever',
 45: 'french_bulldog',
 46: 'german_sheph

In [ ]:
json.dumps(labels_dict)

In [5]:
import json
with open('label_classes.json', 'w') as fp:
    json.dump(labels_dict, fp, indent=4 )

In [6]:
def download_image(url,ext=".jpg"):
    url_hash = hashlib.md5(url.encode('utf-8')).hexdigest()
    file_path = f'{DOWNLOAD_FOLDER}/{url_hash}.{ext}'
    
    urllib.request.urlretrieve(url,file_path)

    img = tf_keras.preprocessing.image.load_img(file_path, target_size=(224, 224))
    return img

def predict(im, model, labels,n=5):
    out = model.predict(im[None,:,:,:]).flatten()
    top_indices = out.argsort()[::-1][:n]
    
    top_probability = out[top_indices]
    top_labels = np.array(labels)[top_indices]
    
    res = list(zip(top_labels,top_probability))
    
    return res

def get_top_n_predictions(preds,labels,n=5):
    if len(preds.shape) >1:
        preds = preds.flatten()
    top_idxs = (-preds).argsort()[:n]
    res =[]
    for idx in top_idxs:
        res.append((labels[idx],preds[idx] ))
    return res

In [7]:
def load_keras_using_tf(model_path):
    custom_objects={'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}
    model = tf_keras.models.load_model(model_path,custom_objects=custom_objects)
    return model

In [8]:
tf_keras_model = load_keras_using_tf('dog_breed_tf.h5')

In [9]:
DOWNLOAD_FOLDER='downloads'
img_url = "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231410/Labrador-Retriever-On-White-01.jpg"
img = download_image(img_url)
img_preprocessed = tf_keras.applications.mobilenet.preprocess_input (
        tf_keras.preprocessing.image.img_to_array(img)
)


In [10]:
predict(im=img_preprocessed ,model=tf_keras_model, labels=labels)

[('labrador_retriever', 0.64543176),
 ('golden_retriever', 0.108133145),
 ('rhodesian_ridgeback', 0.046176903),
 ('great_dane', 0.041474782),
 ('beagle', 0.030086135)]

# convert tflite

In [ ]:
?tf.contrib.lite.TocoConverter.from_keras_model_file

In [ ]:
!ls

In [14]:
with tf_keras.utils.generic_utils.CustomObjectScope({'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}):
    converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed_tf.h5")
    tflite_model = converter.convert()
    open("graph.lite", "wb").write(tflite_model)

INFO:tensorflow:Froze 137 variables.
INFO:tensorflow:Converted 137 variables to const ops.


In [ ]:
?tflite_model

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
?tf.contrib.lite.TocoConverter.from_keras_model_file


In [ ]:
with tf_keras.utils.generic_utils.CustomObjectScope({'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}):
    converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed_tf.h5")
    
    tflite_model = converter.convert()
    open("dog_breed.tflite", "wb").write(tflite_model)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import mobilenet

In [ ]:
from tensorflow.python.keras.utils.generic_utils import CustomObjectScope
from tensorflow.python.keras.applications import mobilenet
from tensorflow.python.keras.models import load_model
with CustomObjectScope({'relu6': mobilenet.relu6,'DepthwiseConv2D': mobilenet.DepthwiseConv2D}):
    model = load_model('dog_breed.h5')
    converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed.h5")
    
    tflite_model = converter.convert()
    open("dog_breed.tflite", "wb").write(tflite_model)

In [ ]:
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model = keras.models.load_model('dog_breed.h5')


Because the Keras model was trained on ImageNet, need to subtract the mean from RGB

- red_bias=-123.68
- green_bias=-116.78
- blue_bias=-103.94

In [ ]:
scale = 1./255
coreml_model = coremltools.converters.keras.convert(model,
                                                    input_names = 'image',
                                                    output_names = 'output',
                                                    image_input_names = 'image',
                                                    image_scale = scale,
                                                    red_bias = -123.68 * scale,
                                                    green_bias = -116.779 * scale,
                                                    blue_bias = -103.939 * scale,
                                                    class_labels=labels)

coreml_model.author = 'Ian Lo'
coreml_model.license = 'MIT'
coreml_model.short_description = 'Model to classify 17 different Singaporean dishes'
coreml_model.input_description['image'] = 'Images from camera in CVPixelBuffer'
coreml_model.output_description['output'] = 'Predicted dishes'

coreml_model.save('dog_breed.mlmodel')

In [ ]:
!ls 

In [ ]:
? converter.convert

In [ ]:
with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model =  load_model('dog_breed.h5')



In [ ]:
import tensorflow as tf

converter = tf.contrib.lite.TocoConverter.from_keras_model_file("dog_breed.h5")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
tf.__version__
